In [4]:
import numpy as np
import optuna
import pandas as pd
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [5]:
train_orig = pd.read_csv('./input/train.csv', index_col='id')
test_orig = pd.read_csv('./input/test.csv', index_col='id')
sample_submission = pd.read_csv('./input/sample_submission.csv')
X_train_orig = pd.DataFrame(train_orig.drop(['defects'], axis=1))
y_train_orig = pd.Series(train_orig.defects)

In [6]:
# Define an objective function for Optuna
def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        "n_estimators": trial.suggest_int("n_estimators", 100, 1200, step=100),
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear','dart']),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
    }
    # Initialize StratifiedKFold for cross-validation
    n_splits = 5  # Number of folds
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize an array to store cross-validated ROC AUC scores
    roc_auc_scores = []

    for train_idx, val_idx in kf.split(X_train_orig, y_train_orig):
        X_train, X_val = X_train_orig.iloc[train_idx], X_train_orig.iloc[val_idx]
        y_train, y_val = y_train_orig.iloc[train_idx], y_train_orig.iloc[val_idx]

        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_val, label=y_val)

        # Train and evaluate the XGBoost model on each fold
        bst = xgb.train(params, dtrain, evals=[(dval, 'eval')], verbose_eval=False)
        y_pred = bst.predict(dval)

        # Calculate ROC AUC score for the fold and append it to the list
        fold_roc_auc = roc_auc_score(y_val, y_pred)
        roc_auc_scores.append(fold_roc_auc)

    # Return the mean ROC AUC score across all folds
    return np.mean(roc_auc_scores)

# Create an Optuna study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and ROC AUC score
best_params = study.best_params
best_roc_auc = study.best_value
print("Best Hyperparameters:", best_params)
print("Best Mean ROC AUC Score:", best_roc_auc)


[I 2023-10-09 09:32:44,706] A new study created in memory with name: no-name-d371deb3-eccd-423a-8eaf-13e50795dbc3
C:\Users\lklen\AppData\Local\Temp\ipykernel_12284\1023974394.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\lklen\AppData\Local\Temp\ipykernel_12284\1023974394.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
C:\Users\lklen\AppData\Local\Temp\ipykernel_12284\1023974394.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.

Best Hyperparameters: {'n_estimators': 700, 'booster': 'gbtree', 'max_depth': 7, 'learning_rate': 0.036716808373589024, 'subsample': 0.7946090501371879, 'colsample_bytree': 0.7966591864261531, 'min_child_weight': 10}
Best Mean ROC AUC Score: 0.7908410598514953


In [8]:
# Use the found hyperparameters to train a model
best_dtrain = xgb.DMatrix(X_train_orig, label=y_train_orig)
best_model = xgb.train(best_params, best_dtrain, num_boost_round=100)

# Make predictions on new data
dtest = xgb.DMatrix(test_orig)
y_pred_proba = best_model.predict(dtest)

submission = pd.Series(data=y_pred_proba, index=test_orig.index, name='defects')
submission.to_csv('solution-v2_3.csv')

C:\Users\lklen\Projects\codeveo\ML\kaggle\ps_se3_ep23\venv\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
C:\Users\lklen\Projects\codeveo\ML\kaggle\ps_se3_ep23\venv\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
C:\Users\lklen\Projects\codeveo\ML\kaggle\ps_se3_ep23\venv\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
C:\Users\lklen\Projects\codeveo\ML\kaggle\ps_se3_ep23\venv\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a